# Python setup

In [5]:
import subprocess 
import time
import glob
import os
from pathlib import Path
import shutil
import pandas as pd
import tempfile
from PIL import Image
import resource
from ast import literal_eval
import json

def run(obj,print_output=False):
    start = time.perf_counter()
    result = subprocess.run(obj["args"],capture_output=True,text=True)    
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        result.check_returncode()
    else:
        if print_output:
            print(result.stdout)
        return { "delta": time.perf_counter() - start, "label": obj["label"],"output": result.stdout,"error": result.stderr, "return_code": result.returncode }
    
rid="linux-x64" #win-x64 #osx-x64
#converter_exe_rel="../../bin/release/net6.0/linux-x64/GraphLogicA"
converter_exe_rel=f"../../src/bin/release/net6.0/{rid}/GraphLogicA"
converter_exe = Path(converter_exe_rel).absolute().as_posix()
graphlogica_exe = converter_exe
minimizer_exe = shutil.which("ltsconvert")
voxlogica_exe = f"../../src/VoxLogicA_1.0-experimental_{rid}/VoxLogicA"
output="output"
#shutil.rmtree(output,ignore_errors=True)
os.makedirs(output,exist_ok=True)
images = glob.glob("test-images/*.png")
    
def mk_df(results,delta_label):
    return pd.DataFrame(results).set_index("label").rename(columns={"delta": delta_label}).drop(columns=["output","error","return_code"])
# !(cd ../.. && dotnet build -c release -r $rid)
resource.setrlimit(resource.RLIMIT_STACK, (resource.RLIM_INFINITY, resource.RLIM_INFINITY))

# Convert images

In [6]:
def converter(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix()], "label": label }
    #return (run(label,args))

converter_df = mk_df([ run(converter(image)) for image in images ],"conversionAndWrite")


# Minimize

In [7]:
def minimizer(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(
        Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    return {"args": [minimizer_exe, "--timings", "-ebranching-bisim", d_path.as_posix(), m_path.as_posix()], "label": label}


out = [run(minimizer(image)) for image in images]

def f(x: str):
    try:
        return float(x)
    except:
        return False

myLabels = { "reachability check": "reachabilityCheck", "total": "mcrl2-int" }

for o in out:
    lines: str = o["error"].strip("- ").splitlines()
    # res = {
    #     x[0]: f(x[1]) for line in lines if (x := line.strip("' ").split(":")) if len(x) > 1 if f(x[1])
    # }
    for line in lines:
        x = line.strip("' ").split(":")
        if len(x) > 1:
            fl = f(x[1])
            l = x[0]
            if l in myLabels.keys():
                l = myLabels[l]
            if fl:
                o[l] = fl

minimizer_df = mk_df(out, "mcrl2")
minimizer_df


,mcrl2,reachabilityCheck,reduction,mcrl2-int
label,,,,
rai-4320,80.120422,3.466415,6.417724,67.519460
maze-1024,2.221477,0.108143,0.212340,2.059682
maze-128,0.036306,0.001322,0.002670,0.029871
maze-2048,8.113061,0.476114,0.836512,8.098491
maze-256,0.127239,0.005717,0.010861,0.118349
maze-4096,33.375868,2.089838,3.275730,33.323664
maze-512,0.499731,0.026499,0.051262,0.490320
maze-8192,136.209284,8.624188,12.902403,135.450699
rai-1080,4.136830,0.200794,0.418057,4.076283


# Convert the minimized model back

In [8]:
def convertBack(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    j_path = o_path.joinpath(path.with_suffix(".json").name)
    return { "args": [converter_exe,"--convert",m_path.as_posix(),j_path.as_posix()], "label": label }
    #return (run(label,args))

backConverter_df = mk_df([ run(convertBack(image)) for image in images ],"convertBack")


# Convert without writing the file 

In [9]:
def fakeConverter(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".fake.aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix(),"--fakeconversion"], "label": label }
    #return (run(label,args))

fakeConverter_df = mk_df([ run(fakeConverter(image)) for image in images ],"conversion")


# Model checking on images using VoxLogicA

In [10]:
def colour(r, g, b, is_graph=False):
    if is_graph:
        return f'''ap("{r:02x}{g:02x}{b:02x}")'''
    else:
        return f'''(red(img) =. {r}) & (green(img) =. {g}) & (blue(img) =. {b})'''


def save(basename, output,form, is_graph):
    p = Path(basename)
    n = p.with_suffix("").name
    if is_graph:
        return f'''save "{output}/{n}_{form}.json" {form}'''
    else:
        return f'''save "{output}/{n}_{form}.png" {form}'''


def mazeSpecification(path, is_graph):
    return f'''
            load img = "{path}"    
            let w = {colour(255,255,255,is_graph)}
            let b = {colour(0,0,255,is_graph)}
            let g = {colour(0,255,0,is_graph)}
            
            let zeta(phi1,phi2) = phi1 | through(N phi1,phi2)
         
            let form1 = zeta(b,w) & zeta(g,w)
            let form2 = b & (!zeta(zeta(g,w),b))
            let form3 = b & (zeta(zeta(g,w),b))
            {save(path,output,"form1",is_graph)}
            {save(path,output,"form2",is_graph)}
            {save(path,output,"form3",is_graph)}
        '''

def raiSpecification(path, is_graph):
    return f'''
            load img="{path}"
            
            let y = {colour(255,255,0,is_graph)}
            let c = {colour(0,255,255,is_graph)}
            let g = {colour(0,255,0,is_graph)}
            let m = {colour(255,0,255,is_graph)}
            let r = {colour(255,0,0,is_graph)}
            let b = {colour(0,0,255,is_graph)}
            let gr = {colour(191,191,191,is_graph)}
            let lgr = {colour(127,127,127,is_graph)}
            let lb = {colour(100,150,255,is_graph)}
            let lg = {colour(0,200,150,is_graph)}
            let lm = {colour(200,50,100,is_graph)}
            let bl = {colour(0,0,0,is_graph)}
            let w = {colour(255,255,255,is_graph)}
            let o = {colour(200,100,0,is_graph)}
            
            let zeta(phi1,phi2) = phi1 | through(N phi1,phi2)
            let ZZ(phi1,phi2) = (!phi2) & zeta(phi2,phi1)
            
            let form1 = y ZZ c ZZ g ZZ m ZZ r ZZ b ZZ gr ZZ bl ZZ w ZZ gr ZZ bl ZZ w ZZ lgr ZZ lb ZZ lg ZZ lm ZZ o
            {save(path,output,"form1",is_graph)}
        '''


def findSpec(img : str,is_graph = False):
    specs = [ ["maze",mazeSpecification],["rai",raiSpecification] ]
    for (prefix,spec) in specs:
        if Path(img).name.startswith(prefix):
            return spec(img,is_graph)
    return None

def modelChecker(image, spec, is_graph=False):
    path = Path(image)
    if is_graph:
        suffix=".grql"
    else:
        suffix=".imgql"
    fname = Path(output).joinpath(path.with_suffix(suffix).name)    
    f = open(fname, "w")
    f.write(spec)
    f.close()
    if is_graph:
        exe = graphlogica_exe
    else:
        exe = voxlogica_exe
    return {"args": [exe, fname], "label": path.with_suffix("").name, "property": "maze"}

modelChecker_df = mk_df(
    [run(modelChecker(image,spec))
        for image in images if (spec:=findSpec(image)) if spec],
    "modelCheckingFull")


# Model Checking on the minimal graph using GraphLogicA

In [11]:
def graph(image):
    path = Path(image)
    o_path = Path(output)
    j_path = o_path.joinpath(path.with_suffix(".json").name)
    return(j_path)

modelCheckerMin_df = mk_df(
    [run(modelChecker(graph(image),spec,True))
        for image in images if (spec:=findSpec(graph(image),True)) if spec],
    "modelCheckingMin")

In [12]:
# Read automata statistics

def autSize(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    first_line = ""
    first_line_min = ""
    with open(d_path,"r") as file:
        first_line = file.readline().lstrip("des ")
    
    with open(m_path,"r") as file:
        first_line_min = file.readline().lstrip("des ")

    autSize = float(os.path.getsize(d_path)) / 1024
    minSize = float(os.path.getsize(m_path)) / 1024

    t = literal_eval(first_line)
    tmin = literal_eval(first_line_min)

    return { "transitions": t[1], "statesMin": tmin[2] , "transitionsMin": tmin[1], "label": label, "autSize": autSize, "minSize": minSize }

autSize_df = pd.DataFrame([ autSize(image) for image in images]).set_index("label")

    

# Gather image sizes and produce the final table

In [13]:
def size(imgpath):    
    path = Path(imgpath)
    img = Image.open(imgpath)
    imgSize = float(os.path.getsize(imgpath)) / 1024
    return { "pixels": img.width * img.height, "label": path.with_suffix("").name, "imgSize": imgSize}

size_df = pd.DataFrame([ size(image) for image in images]).set_index("label")

In [14]:
df = size_df.join(autSize_df).join(fakeConverter_df).join(converter_df).join(minimizer_df).join(backConverter_df).join(modelChecker_df).join(modelCheckerMin_df)
data = df.sort_values(by='pixels')
data

,pixels,imgSize,transitions,statesMin,transitionsMin,autSize,minSize,conversion,conversionAndWrite,mcrl2,reachabilityCheck,reduction,mcrl2-int,convertBack,modelCheckingFull,modelCheckingMin
label,,,,,,,,,,,,,,,,
maze-128,16384,0.527344,145924,7,21,2.531026e+03,0.327148,0.337745,0.337722,0.036306,0.001322,0.002670,0.029871,0.358192,0.485221,0.386993
rai-130,31200,1.473633,278584,155,899,4.943358e+03,15.409180,0.321588,0.380107,0.067037,0.002601,0.004931,0.057655,0.288516,0.516808,0.443175
maze-256,65536,1.100586,586756,7,21,1.060308e+04,0.327148,0.409222,0.430382,0.127239,0.005717,0.010861,0.118349,0.293551,0.463278,0.438237
rai-260,124800,3.340820,1118764,315,1841,2.075891e+04,32.987305,0.311347,0.620439,0.262961,0.012273,0.021885,0.256452,0.319955,0.535636,0.492792
maze-512,262144,3.356445,2353156,7,21,4.562278e+04,0.327148,0.336035,0.779174,0.499731,0.026499,0.051262,0.490320,0.297641,0.454684,0.469474
rai-540,518400,10.313477,4656604,460,2766,9.250192e+04,49.872070,0.347735,0.903518,1.087049,0.050022,0.106193,1.008549,0.304475,0.776253,0.510545
maze-1024,1048576,12.325195,9424900,7,21,1.887647e+05,0.327148,0.391419,1.284526,2.221477,0.108143,0.212340,2.059682,0.307182,0.509390,0.379357
rai-1080,2073600,28.714844,18644404,945,6965,3.935059e+05,127.243164,0.397260,4.995304,4.136830,0.200794,0.418057,4.076283,0.317041,1.572300,0.571500
maze-2048,4194304,41.973633,37724164,7,21,8.127831e+05,0.327148,0.461486,4.121310,8.113061,0.476114,0.836512,8.098491,0.343321,0.821224,0.413967


# Export data

In [15]:
df.to_csv("rawdata.csv")


For batch execution: 
    
    rm output -rf && jupyter nbconvert --to script --ExecutePreprocessor.timeout=18000 --execute testMin.ipynb